# Capstone DS Course
## Emet Flores
### Week 05 - Final Assignment
### Part 02_2_1 - Review obtained data fro Zip-Codes.com
### Notes:
 - Additional geo-demographic data for San Diego Zip Codes
 - Using Zip-Codes.com API
 https://www.zip-codes.com/zip-code-api.asp
 - Using dataset obtained from their API

In [1]:
# Import necesary libraries
import pandas as pd
import numpy as np # library for vectorized computation
from sklearn.cluster import KMeans  #for modeling clusters


In [36]:
# Get the additional data that we processed and documented in a CSV file

San_Diego_codes = pd.read_csv ('SD_Detail_Data_Zipcodes.csv', sep = ',', index_col=0)

San_Diego_codes.head()

,ZipCode,ZipCodePopulation,HouseholdsPerZipcode,WhitePop,BlackPop,HispanicPop,AsianPop,IndianPop,HawaiianPop,OtherPop,...,AverageHouseValue,IncomePerHousehold,MedianAge,AverageFamilySize,Latitude,Longitude,AreaLand,AreaWater,City,CountyName
0,90210,21741,8669,19761,471,1092,1966,80,55,387,...,2000000.00,149732.00,47.50,3.06,34.10,-118.42,10.13,0.06,BEVERLY HILLS,LOS ANGELES
1,91901,17403,6345,15466,315,2644,564,743,101,856,...,525700.00,90397.00,41.90,3.10,32.79,-116.71,89.26,0.78,ALPINE,SAN DIEGO
2,91902,17653,5956,12379,757,7326,2481,272,217,2596,...,607100.00,92759.00,43.20,3.26,32.68,-117.01,8.71,0.30,BONITA,SAN DIEGO
3,91905,1700,632,1335,76,389,25,189,20,133,...,279900.00,32819.00,46.00,3.06,32.73,-116.30,65.81,0.02,BOULEVARD,SAN DIEGO
4,91906,3627,1254,2828,175,1059,66,319,15,409,...,221300.00,49919.00,37.20,3.23,32.70,-116.50,105.53,0.71,CAMPO,SAN DIEGO


In [37]:
# Count the number of target Zip codes 
San_Diego_codes.shape

(108, 23)

In [38]:
# We remove the demo Zipcode from our dataset

San_Diego_codes.drop(San_Diego_codes.index[0],inplace=True)
San_Diego_codes.head(5)

,ZipCode,ZipCodePopulation,HouseholdsPerZipcode,WhitePop,BlackPop,HispanicPop,AsianPop,IndianPop,HawaiianPop,OtherPop,...,AverageHouseValue,IncomePerHousehold,MedianAge,AverageFamilySize,Latitude,Longitude,AreaLand,AreaWater,City,CountyName
1,91901,17403,6345,15466,315,2644,564,743,101,856,...,525700.00,90397.00,41.90,3.10,32.79,-116.71,89.26,0.78,ALPINE,SAN DIEGO
2,91902,17653,5956,12379,757,7326,2481,272,217,2596,...,607100.00,92759.00,43.20,3.26,32.68,-117.01,8.71,0.30,BONITA,SAN DIEGO
3,91905,1700,632,1335,76,389,25,189,20,133,...,279900.00,32819.00,46.00,3.06,32.73,-116.30,65.81,0.02,BOULEVARD,SAN DIEGO
4,91906,3627,1254,2828,175,1059,66,319,15,409,...,221300.00,49919.00,37.20,3.23,32.70,-116.50,105.53,0.71,CAMPO,SAN DIEGO
5,91910,75802,26063,47051,4255,45275,9351,1229,786,17635,...,424700.00,59371.00,35.60,3.40,32.64,-117.05,12.23,0.03,CHULA VISTA,SAN DIEGO


In [39]:
# Count the number of target Zip codes 
San_Diego_codes.shape

(107, 23)

In [40]:
# Import additional libraries for data vizualization

import numpy as np
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

print('Required libraries imported.')

Required libraries imported.


# Review our available -San Diego- geographic data on a Map

In [41]:
address = 'San Diego, CA'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Diego is {}, {}.'.format(latitude, longitude))


The geograpical coordinate of San Diego is 32.7174209, -117.1627714.


In [42]:
# create map of San Diego using latitude and longitude values
map_SD = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, city, zipcode in zip(San_Diego_codes['Latitude'], San_Diego_codes['Longitude'], San_Diego_codes['City'], San_Diego_codes['ZipCode']):
    label = '{}, {}'.format(zipcode, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SD)  
    
map_SD

# Review if there are any relevant clusters
# (Demographic data)
## Normalizing and using K-Means

In [43]:
# Copy our dataframe and
# Remove columns that will not provide insight
sd_data = San_Diego_codes.drop(['ZipCode','Latitude','Longitude','City','CountyName'], axis=1)
sd_data.head()

,ZipCodePopulation,HouseholdsPerZipcode,WhitePop,BlackPop,HispanicPop,AsianPop,IndianPop,HawaiianPop,OtherPop,MalePop,FemalePop,PersonsPerHousehold,AverageHouseValue,IncomePerHousehold,MedianAge,AverageFamilySize,AreaLand,AreaWater
1,17403,6345,15466,315,2644,564,743,101,856,8750,8653,2.70,525700.00,90397.00,41.90,3.10,89.26,0.78
2,17653,5956,12379,757,7326,2481,272,217,2596,8603,9050,2.96,607100.00,92759.00,43.20,3.26,8.71,0.30
3,1700,632,1335,76,389,25,189,20,133,965,735,2.51,279900.00,32819.00,46.00,3.06,65.81,0.02
4,3627,1254,2828,175,1059,66,319,15,409,1959,1668,2.74,221300.00,49919.00,37.20,3.23,105.53,0.71
5,75802,26063,47051,4255,45275,9351,1229,786,17635,36862,38940,2.87,424700.00,59371.00,35.60,3.40,12.23,0.03


In [44]:
# And normalize the data

from sklearn.preprocessing import StandardScaler

X = sd_data.values[:,1:]
X = np.nan_to_num(X)
sd_cluster_dataset = StandardScaler().fit_transform(X)
sd_cluster_dataset

array([[-0.50141176, -0.29628279, -0.59193777, ...,  0.03806075,
         0.75729162,  0.64412801],
       [-0.55258004, -0.51058762, -0.41738838, ...,  0.25311025,
        -0.31730389, -0.07627154],
       [-1.25288831, -1.27728102, -0.68632082, ..., -0.01570163,
         0.44450714, -0.48558947],
       ...,
       [ 1.32131019,  1.53339588,  1.41695993, ...,  1.15363006,
         0.05957959,  0.61882472],
       [-1.33602032, -1.34246801, -0.68474119, ..., -4.12852343,
        -0.430481  , -0.51833491],
       [-0.34119849, -0.11946569, -0.49794964, ...,  1.44932313,
        -0.36596845, -0.213207  ]])

In [45]:
# And review if we can osberve some relevant clusters within the data
num_clusters = 5

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(sd_cluster_dataset)
labels = k_means.labels_

print(labels)

[2 2 3 3 0 0 4 2 4 3 3 3 4 3 2 4 4 4 3 0 3 3 0 3 3 3 3 2 4 4 2 2 2 4 0 0 4
 0 4 0 4 2 3 4 4 4 3 0 0 0 3 3 3 4 4 3 2 0 3 4 2 4 4 4 0 0 3 2 4 0 4 4 0 2
 4 4 4 4 4 0 0 0 4 4 2 4 4 2 4 4 4 0 4 4 4 4 4 1 1 1 0 3 2 1 0 1 4]


In [46]:
# And asociate the cluster information to our dataset
San_Diego_codes["Labels"] = labels
San_Diego_codes.head(5)

,ZipCode,ZipCodePopulation,HouseholdsPerZipcode,WhitePop,BlackPop,HispanicPop,AsianPop,IndianPop,HawaiianPop,OtherPop,...,IncomePerHousehold,MedianAge,AverageFamilySize,Latitude,Longitude,AreaLand,AreaWater,City,CountyName,Labels
1,91901,17403,6345,15466,315,2644,564,743,101,856,...,90397.00,41.90,3.10,32.79,-116.71,89.26,0.78,ALPINE,SAN DIEGO,2
2,91902,17653,5956,12379,757,7326,2481,272,217,2596,...,92759.00,43.20,3.26,32.68,-117.01,8.71,0.30,BONITA,SAN DIEGO,2
3,91905,1700,632,1335,76,389,25,189,20,133,...,32819.00,46.00,3.06,32.73,-116.30,65.81,0.02,BOULEVARD,SAN DIEGO,3
4,91906,3627,1254,2828,175,1059,66,319,15,409,...,49919.00,37.20,3.23,32.70,-116.50,105.53,0.71,CAMPO,SAN DIEGO,3
5,91910,75802,26063,47051,4255,45275,9351,1229,786,17635,...,59371.00,35.60,3.40,32.64,-117.05,12.23,0.03,CHULA VISTA,SAN DIEGO,0


## And visualize the clusters

In [47]:
# Lets reuse code from a previous lab
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(San_Diego_codes['Latitude'], San_Diego_codes['Longitude'], San_Diego_codes['ZipCode'], San_Diego_codes['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Initial Observations - Zip-Codes.com API
## At a glance:
- Cluster 0 (Red): This cluster describes **LOWER INCOME** households.
- Cluster 1 (Purple): This cluster provides no relevant insight, relvant for naval offices.
- Cluster 2 (Light Blue): This cluster describes **AFLUENT** households.
- Cluster 3 (Light Green): This cluster describes low density areas, moslty outside our scope of interest
- Cluster 4 (Orange): This cluster is the most common with **TYPICAL** households.

## Note: If you run the cluster algorithms again, the order of the documented clusters changes.

In [48]:
# We update pandas to more format friendly for reading
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [49]:
San_Diego_codes.loc[San_Diego_codes['Labels'] == 0,San_Diego_codes.columns[[1] + list(range(2, San_Diego_codes.shape[1]-7))]].describe()

,ZipCodePopulation,HouseholdsPerZipcode,WhitePop,BlackPop,HispanicPop,AsianPop,IndianPop,HawaiianPop,OtherPop,MalePop,FemalePop,PersonsPerHousehold,AverageHouseValue,IncomePerHousehold,MedianAge,AverageFamilySize
count,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00
mean,57493.86,17533.77,32439.73,5136.45,27073.64,8137.09,984.09,729.05,13519.00,28756.86,28737.00,3.22,390495.45,56671.41,31.99,3.64
std,13271.45,4461.15,10120.81,3633.39,13181.03,7513.83,201.42,301.54,6036.00,6654.24,6748.09,0.35,53790.73,12394.30,3.35,0.28
min,35125.00,10216.00,12288.00,1475.00,10197.00,1815.00,473.00,246.00,5103.00,16905.00,18131.00,2.61,299400.00,33125.00,24.60,3.19
25%,48235.00,14153.50,25195.75,2792.50,16998.25,3481.00,842.00,519.00,8462.50,24166.50,24067.75,3.04,348250.00,48718.00,30.27,3.43
50%,56916.50,17457.50,32076.00,4289.50,23360.00,5511.50,1033.00,727.50,12734.50,28440.50,28502.00,3.23,369450.00,57793.00,32.85,3.68
75%,65341.75,20455.50,38476.50,5470.25,35204.75,11341.25,1128.25,930.25,17154.50,32023.50,33318.25,3.39,435575.00,61545.50,33.85,3.84
max,82999.00,26063.00,50669.00,16789.00,58816.00,35353.00,1302.00,1292.00,24733.00,41254.00,43061.00,4.00,478500.00,92061.00,38.40,4.28


In [50]:
San_Diego_codes.loc[San_Diego_codes['Labels'] == 1,San_Diego_codes.columns[[1] + list(range(2, San_Diego_codes.shape[1]-7))]].describe()

,ZipCodePopulation,HouseholdsPerZipcode,WhitePop,BlackPop,HispanicPop,AsianPop,IndianPop,HawaiianPop,OtherPop,MalePop,FemalePop,PersonsPerHousehold,AverageHouseValue,IncomePerHousehold,MedianAge,AverageFamilySize
count,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
mean,375.60,2.80,288.40,49.00,56.40,23.40,7.00,2.20,23.00,325.20,50.40,0.60,0.00,0.00,18.20,0.40
std,262.13,5.72,210.32,31.46,39.22,13.99,5.10,2.28,17.00,231.43,38.76,0.89,0.00,0.00,10.19,0.89
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,198.00,0.00,131.00,37.00,34.00,24.00,6.00,0.00,15.00,161.00,37.00,0.00,0.00,0.00,22.00,0.00
50%,550.00,0.00,396.00,63.00,74.00,25.00,6.00,2.00,26.00,455.00,42.00,0.00,0.00,0.00,22.40,0.00
75%,559.00,1.00,447.00,65.00,75.00,32.00,9.00,4.00,28.00,502.00,69.00,1.00,0.00,0.00,23.10,0.00
max,571.00,13.00,468.00,80.00,99.00,36.00,14.00,5.00,46.00,508.00,104.00,2.00,0.00,0.00,23.50,2.00


In [51]:
San_Diego_codes.loc[San_Diego_codes['Labels'] == 2,San_Diego_codes.columns[[1] + list(range(2, San_Diego_codes.shape[1]-7))]].describe()

,ZipCodePopulation,HouseholdsPerZipcode,WhitePop,BlackPop,HispanicPop,AsianPop,IndianPop,HawaiianPop,OtherPop,MalePop,FemalePop,PersonsPerHousehold,AverageHouseValue,IncomePerHousehold,MedianAge,AverageFamilySize
count,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00
mean,13105.69,4879.94,10886.44,404.50,2364.56,1282.19,192.19,82.12,852.44,6585.88,6519.81,2.64,933062.50,109993.12,41.66,3.04
std,6919.22,2546.69,5952.76,464.88,2196.49,1063.41,183.30,68.22,781.43,3604.52,3379.21,0.48,415075.12,17789.92,7.52,0.34
min,1048.00,390.00,846.00,12.00,40.00,50.00,3.00,0.00,9.00,476.00,572.00,1.91,525700.00,85363.00,30.20,2.46
25%,9307.25,3330.50,8536.50,96.50,656.25,552.00,90.00,32.25,179.00,4616.25,4691.00,2.30,603400.00,99104.50,37.95,2.88
50%,13768.00,5382.00,11412.50,247.50,1987.00,1000.00,157.00,67.50,779.50,6729.50,7038.50,2.56,827100.00,103780.50,42.15,2.99
75%,17995.00,6523.00,15492.00,493.50,2778.25,1662.75,249.50,109.50,912.25,8885.50,9152.25,2.80,1153900.00,129201.50,44.22,3.13
max,23575.00,9034.00,20030.00,1815.00,7326.00,4153.00,743.00,217.00,2596.00,13532.00,11705.00,3.72,2000000.00,141691.00,59.00,3.78


In [52]:
San_Diego_codes.loc[San_Diego_codes['Labels'] == 3,San_Diego_codes.columns[[1] + list(range(2, San_Diego_codes.shape[1]-7))]].describe()

,ZipCodePopulation,HouseholdsPerZipcode,WhitePop,BlackPop,HispanicPop,AsianPop,IndianPop,HawaiianPop,OtherPop,MalePop,FemalePop,PersonsPerHousehold,AverageHouseValue,IncomePerHousehold,MedianAge,AverageFamilySize
count,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00,22.00
mean,2720.95,719.27,2147.50,129.55,665.00,97.55,152.73,23.59,280.91,1740.14,980.82,2.66,238368.18,43147.00,43.32,3.10
std,3519.89,742.31,2961.43,292.59,760.71,189.49,202.35,51.78,320.88,3007.95,1034.97,0.58,161348.10,31711.85,10.48,0.49
min,78.00,7.00,76.00,0.00,1.00,0.00,0.00,0.00,1.00,36.00,28.00,1.77,0.00,0.00,20.00,2.38
25%,800.75,257.75,545.25,10.25,159.25,11.00,29.50,0.50,64.25,411.75,305.00,2.29,132950.00,26354.00,37.38,2.75
50%,1620.00,506.50,1276.50,20.00,422.50,27.50,74.50,8.50,165.50,811.50,764.50,2.48,278150.00,41411.50,44.45,3.05
75%,3438.75,835.50,2883.50,70.75,862.00,48.75,179.00,15.75,430.75,1896.00,1343.25,2.90,364650.00,66029.50,50.75,3.29
max,15655.00,3024.00,13505.00,1086.00,2857.00,706.00,738.00,228.00,1380.00,14267.00,4549.00,4.02,589100.00,104760.00,60.80,4.33


In [53]:
San_Diego_codes.loc[San_Diego_codes['Labels'] == 4,San_Diego_codes.columns[[1] + list(range(2, San_Diego_codes.shape[1]-7))]].describe()

,ZipCodePopulation,HouseholdsPerZipcode,WhitePop,BlackPop,HispanicPop,AsianPop,IndianPop,HawaiianPop,OtherPop,MalePop,FemalePop,PersonsPerHousehold,AverageHouseValue,IncomePerHousehold,MedianAge,AverageFamilySize
count,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00
mean,37039.60,14455.48,27975.71,1702.88,8132.29,4905.17,586.02,302.98,3595.26,18396.79,18642.81,2.58,572345.24,81270.00,36.56,3.09
std,9848.14,4315.38,8530.10,1111.74,5212.42,4296.69,307.58,133.41,2575.36,4899.85,5079.11,0.48,179982.58,22489.75,3.89,0.34
min,18858.00,6209.00,13808.00,227.00,2765.00,723.00,252.00,97.00,836.00,9446.00,9411.00,1.47,326500.00,38911.00,29.20,2.33
25%,27899.50,10987.75,21414.75,932.25,4341.50,2256.50,374.50,203.50,1545.75,13954.00,13774.00,2.24,461775.00,65507.50,33.68,2.90
50%,38703.00,15160.50,27327.50,1395.00,6723.50,2913.00,518.00,281.00,3098.50,19487.50,19925.50,2.63,539400.00,77251.50,37.25,3.12
75%,45614.25,17398.50,34975.50,2023.00,10494.50,6054.25,720.75,375.75,5000.75,22603.25,22764.50,2.84,647750.00,88011.50,39.60,3.27
max,53422.00,23349.00,46690.00,4932.00,27436.00,17682.00,1645.00,652.00,10551.00,25877.00,27596.00,3.88,1276500.00,141926.00,43.00,4.15


In [55]:
# And finaly save our target Zipcodes,
# removing all Zipcode references that will not be useful
# keep the 3 clusters that provide relevant insight
# and verify the cluster numbering so we filter the target zipcodes correctly


export_file = San_Diego_codes.loc[San_Diego_codes['Labels'].isin([0,2,4])]

export_file.head(5)

,ZipCode,ZipCodePopulation,HouseholdsPerZipcode,WhitePop,BlackPop,HispanicPop,AsianPop,IndianPop,HawaiianPop,OtherPop,...,IncomePerHousehold,MedianAge,AverageFamilySize,Latitude,Longitude,AreaLand,AreaWater,City,CountyName,Labels
1,91901,17403,6345,15466,315,2644,564,743,101,856,...,90397.00,41.90,3.10,32.79,-116.71,89.26,0.78,ALPINE,SAN DIEGO,2
2,91902,17653,5956,12379,757,7326,2481,272,217,2596,...,92759.00,43.20,3.26,32.68,-117.01,8.71,0.30,BONITA,SAN DIEGO,2
5,91910,75802,26063,47051,4255,45275,9351,1229,786,17635,...,59371.00,35.60,3.40,32.64,-117.05,12.23,0.03,CHULA VISTA,SAN DIEGO,0
6,91911,82999,24622,48709,4063,58816,8051,1208,955,24733,...,52274.00,33.50,3.72,32.61,-117.05,11.71,0.19,CHULA VISTA,SAN DIEGO,0
7,91913,40971,12133,21126,3018,18109,12528,428,532,6313,...,87440.00,32.00,3.67,32.63,-116.99,10.20,0.04,CHULA VISTA,SAN DIEGO,4


In [56]:
# And review the amount of records
export_file.shape

(80, 24)

In [57]:
# Create a simple CSV File with the target zip codes

export_file.to_csv('SD_Target_Zipcodes.csv', sep = ',', encoding = 'ANSI')